In [2]:
import os

import sentencepiece as spm

from utils.clean import get_azerbaijani_dataset
from utils.config import ROOT_DIR

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hikme\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
TOKENIZER_PATH = f"{ROOT_DIR}/tokenizer"

os.makedirs(TOKENIZER_PATH, exist_ok=True)

In [4]:
DATASET_NAMES = ("LocalDoc/AzTC", "allmalab/DOLLMA")

get_azerbaijani_dataset(DATASET_NAMES)

2025-08-15 00:01:58,892 - utils.clean - INFO - Pruned content of data/azerbaijani_data.txt
2025-08-15 00:01:58,894 - utils.clean - INFO - Pruned content of data/LocalDoc_AzTC.txt
2025-08-15 00:01:58,895 - utils.clean - INFO - Downloading LocalDoc/AzTC
2025-08-15 00:02:29,280 - utils.clean - INFO -   Downloading config: default
Generating train split: 100%|██████████| 51477480/51477480 [00:27<00:00, 1849254.03 examples/s]
2025-08-15 00:16:45,851 - utils.clean - INFO - Successfully wrote text data to data/LocalDoc_AzTC.txt
2025-08-15 00:16:45,863 - utils.clean - INFO - Pruned content of data/allmalab_DOLLMA.txt
2025-08-15 00:16:45,864 - utils.clean - INFO - Downloading allmalab/DOLLMA
2025-08-15 00:17:00,847 - utils.clean - INFO -   Downloading config: anl-news
Generating train split: 100%|██████████| 236443/236443 [00:03<00:00, 77962.27 examples/s] 
2025-08-15 00:17:21,050 - utils.clean - INFO -   Downloading config: azwiki
Generating train split: 100%|██████████| 129433/129433 [00:00<0

In [5]:
options = dict(
  # Input spec
  input=f"{ROOT_DIR}/data/azerbaijani_data.txt",
  input_format="text",
  # Output spec
  model_prefix=f"{TOKENIZER_PATH}/azerbaijani_bpe",
  # Algorithm spec
  model_type="bpe",
  vocab_size=40_000,
  # Normalization
  normalization_rule_name="identity", # No normalization applied
  remove_extra_whitespaces=False,
  input_sentence_size=50_000_000, # Set to 0 zero to train on the whole dataset
  max_sentence_length=4192, # Max number of bytes per sentence
  seed_sentencepiece_size=1_000_000,
  shuffle_input_sentence=True,
  # Rare word handling
  character_coverage=0.9995,
  byte_fallback=True,
  # Merge rules
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True,
  allow_whitespace_only_pieces=True,
  # Special tokens
  unk_id=0, # the UNK token must exist
  bos_id=1, # The others are optional, set to -1 to turn off
  eos_id=2,
  pad_id=-1,
  # Systems
  num_threads=os.cpu_count()
)

spm.SentencePieceTrainer.train(**options)

In [6]:
sp = spm.SentencePieceProcessor()
sp.load(f"{TOKENIZER_PATH}/azerbaijani_bpe.model")
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]

In [7]:
azerbaijani_samples = [
        "Kitab deyəndə burda mədəniyyət, incəsənət nümunələri nəzərdə tuturam.",
        "Uzun müddətdir ki, cəbhədə rəsmi olaraq atəşkəs hökm sürür.",
        "Biz Bütöv Azərbaycançıyıq, bunu öz əməllərimizdə göstərmişik."
]

In [8]:
for sentence in azerbaijani_samples:
    ids = sp.encode(sentence)
    print([sp.id_to_piece(idx) for idx in ids])

['▁Kitab', '▁deyəndə', '▁burda', '▁mədəniyyət', ',', '▁incəsənət', '▁nümunələri', '▁nəzərdə', '▁tuturam', '.']
['▁Uzun', '▁müddətdir', '▁ki', ',', '▁cəbhədə', '▁rəsmi', '▁olaraq', '▁atəşkəs', '▁hökm', '▁sürür', '.']
['▁Biz', '▁Bütöv', '▁Azərbaycan', 'çı', 'yıq', ',', '▁bunu', '▁öz', '▁əməl', 'lərimizdə', '▁göstər', 'mişik', '.']
